# Examining the effect of including glacial runoff in SPEI drought risk calculations

Welcome!  This notebook will reproduce and visualise the analyses behind Ultee & Coats, "Glacial runoff modulates 21st century basin-level water availability, but models disagree on the details" (submitted to GRL), using helper functions from gSPEI.py.  All code is stored in a public GitHub repository--click [here](https://github.com/ehultee/glacial-SPEI) for access.

### Loading in modules and data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from gSPEI import *

In [ ]:
fpath = '/Users/lizz/Documents/GitHub/glacial-SPEI/data/SPEI_Files/' # file path to SPEI output files

## Settings in filenames
integration_times = np.arange(3, 28, 4) # all SPEI integration times used
modelnames = ['CanESM2', 'CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GISS-E2-R', 'INMCM4', 'MIROC-ESM', 'NorESM1-M'] # all models used in comparison
scenarios = ['Rcp4p5', 'Rcp8p5'] # climate scenarios

## Basins in the order they are written in each file
basins = ['INDUS','TARIM','BRAHMAPUTRA','ARAL SEA','COPPER','GANGES','YUKON','ALSEK','SUSITNA','BALKHASH','STIKINE','SANTA CRUZ',
'FRASER','BAKER','YANGTZE','SALWEEN','COLUMBIA','ISSYK-KUL','AMAZON','COLORADO','TAKU','MACKENZIE','NASS','THJORSA','JOEKULSA A F.',
'KUSKOKWIM','RHONE','SKEENA','OB','OELFUSA','MEKONG','DANUBE','NELSON RIVER','PO','KAMCHATKA','RHINE','GLOMA','HUANG HE','INDIGIRKA',
'LULE','RAPEL','SANTA','SKAGIT','KUBAN','TITICACA','NUSHAGAK','BIOBIO','IRRAWADDY','NEGRO','MAJES','CLUTHA','DAULE-VINCES',
'KALIXAELVEN','MAGDALENA','DRAMSELV','COLVILLE']

Now we will load data into a dictionary for one specific climate scenario, indexed by model name.  You can choose to analyse a different climate scenario or a different SPEI integration timescale by changing the arguments `itime` and `scen` below.

In [ ]:
itime = integration_times[3] # select timescale of integration. [3] is 15 months, compatible with PDSI
scen = scenarios[0] # choose whether to load RCP 4.5 or RCP 8.5

SPEI_by_model = {m: {} for m in modelnames} # create dictionary indexed by model name
for m in modelnames:
    norunoff_f_m = fpath+'NRunoff_{}_{}_{}.txt'.format(itime, m, scen)
    wrunoff_f_m = fpath+'WRunoff_{}_{}_{}.txt'.format(itime, m, scen)
    SPEI_by_model[m]['NRunoff'] = np.loadtxt(norunoff_f_m)
    SPEI_by_model[m]['WRunoff'] = np.loadtxt(wrunoff_f_m)
    SPEI_by_model[m]['diff'] = SPEI_by_model[m]['WRunoff'] - SPEI_by_model[m]['NRunoff']

## Plot SPEI time series for a given basin

First, we can plot time series of SPEI with and without glacial runoff included.  Examples of this kind of figure appear in Ultee & Coats Figure 1.

We will use a helper function from gSPEI.py to compare SPEI running mean with no glacial runoff (orange) versus with glacial runoff (blue).  The function plots SPEI time series for a single basin, separated by the Earth System Model that produced each series.  Choose the basin by its index in the `basins` list, above. 

By default, we plot a running mean so that long-term trends can be more easily seen.  If you prefer, you can adjust `window` to show time series with less smoothing (shorter rolling-mean windows).

In [ ]:
basin_id = 1 # index of basin of interest in "basins", above
window = 30 # window in years over which to calculate the running mean

In [ ]:
plot_runmean_comparison(basin_id=1, permodel_dict=SPEI_by_model, window_yrs=window)

### Isolating the glacial effect on SPEI mean
We will use another helper function from gSPEI to plot the running mean glacial effect in the same basin.  This function produces the panels seen in Ultee & Coats Figure 2 (four example basins) and Supplementary Figure S1 (all basins).

In [ ]:
plot_basin_runmean(basin_id=basin_id, permodel_dict=SPEI_by_model, window_yrs=window)

### Isolating the glacial effect on running SPEI variance
Similarly, we can plot the glacial effect on the 30-year running variance in the same basin.  This function produces the panels seen in Ultee & Coats Figure 3 (four example basins) and Supplementary Figure S2 (all basins).

Adjust the window size and the basin index in the cell above, then re-run the cells in this section, to see the mean and variance changes for the particular case that interests you.

In [ ]:
plot_basin_runvar(basin_id=basin_id, permodel_dict=SPEI_by_model, window_yrs=window)

## Multi-basin summary: change in SPEI at end of century

The functions above show timeseries and running statistics for individual basins.  But what can be said in general?  When we account for glacial runoff in our SPEI calculation, how does the mean SPEI change?  Does SPEI become more or less variable?

We calculate the mean SPEI with and without glacial runoff, for each of our 8 climate models, in each of our 56 basins, over a 30-year period.  We do the same for variance.  Then, we plot markers with whiskers to visualize the inter-model range in each.  

Figure 4 of Ultee & Coats examines these summary results for the 30-year period at the end of the 21st century, but you can explore other periods by adjusting `yrs` below.  If you'd like to examine a different climate scenario, you can return to the top of the notebook and change the climate scenario setting `scen`.

In [ ]:
## Calculate changes due to glacial effect at end of century, using gSPEI functions
yrs = (2070, 2100)

bas_glac_meanmed, mean_spread = glacial_meandiff(SPEI_by_model, years=yrs)
bas_glac_varmed, var_spread = glacial_vardiff(SPEI_by_model, years=yrs)

plt.figure('Mean and variance shifts due to glacial effects in 2070-2100')
plt.errorbar(x=bas_glac_meanmed, y=bas_glac_varmed, xerr=mean_spread, yerr=var_spread, ls='', marker='d', elinewidth=2.0, color='DarkBlue')
plt.axes().set_xlabel('Difference in mean SPEI', fontsize=16)
plt.axes().set_ylabel('Difference in SPEI variance', fontsize=16)
plt.axes().set_ylim(-1.5, 1.0)
plt.axes().set_xlim(-0.5, 5)
plt.show()

## Bonus: SPEI shifts over time
SPEI responds to changes in global and regional climate over time.  So, it may be of interest to compare 30-year means from a period in the 20th century versus a period later in the 21st century.  This preliminary analysis was not included in the Ultee & Coats manuscript.

We compute single-model mean and variance of SPEI for each basin, for two 30-year periods.  Below, we choose the historical period 1950-1980, before glacial runoff from the Huss & Hock model is introduced (1980), and the future period 2070-2100, at the end of the 21st century.  The plot is in the same style as the one above.  Remember, this plot shows shift in SPEI statistics over time rather than the shift in a single time period due to glacial effects.

In [ ]:
## 30-yr period means and variance
modelmeans_1950_1980 = {m: [] for m in modelnames} #dictionary indexed by model name
modelvar_1950_1980 = {m: [] for m in modelnames}
modelmeans_2070_2100 = {m: [] for m in modelnames}
modelvar_2070_2100 = {m: [] for m in modelnames}
mean_shifts = {m: [] for m in modelnames}
var_shifts = {m: [] for m in modelnames}

for m in modelnames:
    means_i = [np.nanmean(SPEI_by_model[m]['diff'][j][600:971]) for j in range(len(basin_names))]
    var_i = [np.nanvar(SPEI_by_model[m]['diff'][j][600:971]) for j in range(len(basin_names))]
    means_f = [np.nanmean(SPEI_by_model[m]['diff'][j][2039:2410]) for j in range(len(basin_names))]
    var_f = [np.nanvar(SPEI_by_model[m]['diff'][j][2039:2410]) for j in range(len(basin_names))]
    modelmeans_1950_1980[m] = means_i
    modelvar_1950_1980[m] = var_i
    modelmeans_2070_2100[m] = means_f
    modelvar_2070_2100[m] = var_f
    mean_shifts[m] = np.array(means_f) - np.array(means_i)
    var_shifts[m] = np.array(var_f) - np.array(var_i)

basin_mean_shifts = {b: [] for b in basin_names} #dictionary indexed by basin name
basin_var_shifts = {b: [] for b in basin_names} #dictionary indexed by model name
basin_meanshift_meds = [] #arrays for plotting
basin_varshift_meds = []
basin_meanshift_range = []
basin_varshift_range = []

for i, b in enumerate(basin_names):
    bmeans_i = [np.nanmean(SPEI_by_model[m]['WRunoff'][i][600:971]) for m in modelnames]
    bvar_i = [np.nanvar(SPEI_by_model[m]['WRunoff'][i][600:971]) for m in modelnames]
    bmeans_f = [np.nanmean(SPEI_by_model[m]['WRunoff'][i][2039:2410]) for m in modelnames]
    bvar_f = [np.nanvar(SPEI_by_model[m]['WRunoff'][i][2039:2410]) for m in modelnames]
    basin_mean_shifts[b] = np.array(bmeans_f) - np.array(bmeans_i)
    basin_var_shifts[b] = np.array(bvar_f) - np.array(bvar_i)
    basin_meanshift_meds.append(np.nanmedian(basin_mean_shifts[b]))
    basin_varshift_meds.append(np.nanmedian(basin_var_shifts[b]))
    basin_meanshift_range.append(np.nanmax(basin_mean_shifts[b]) - np.nanmin(basin_mean_shifts[b]))
    basin_varshift_range.append(np.nanmax(basin_var_shifts[b]) - np.nanmin(basin_var_shifts[b]))


plt.figure('Mean and variance shifts, 2070-2100 versus 1950-1980, per basin for WRunoff case')
plt.errorbar(x=basin_meanshift_meds, y=basin_varshift_meds, xerr=basin_meanshift_range, yerr=basin_varshift_range, ls='')
plt.axes().set_xlabel('Difference in 30-yr mean SPEI', fontsize=16)
plt.axes().set_ylabel('Difference in SPEI variance', fontsize=16)
plt.axes().set_ylim(-0.5, 2.5)
plt.axes().set_xlim(-1, 5)
plt.show()